In [175]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [176]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [177]:
data.drop(["RowNumber", "CustomerId", "Surname"], axis = 1, inplace = True)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Applying Labelencoder for Gender

In [178]:
encoder = LabelEncoder()
cols = ["Gender"]
for col in cols:
    data[col] = encoder.fit_transform(data[col])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


### Applying one_hot_encoder for Geography

In [179]:
onehotencoder = OneHotEncoder()
onehotencoded = onehotencoder.fit_transform(data[["Geography"]])
onehotencoded

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

Seeing the features which are one hot encoder

In [180]:
features_of_one_hot = onehotencoder.get_feature_names_out(["Geography"])
features_of_one_hot

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

### Converting the one_hot_data into dataframe with column names as features(It's done only after coverting data into array)

In [181]:
one_hot_data = pd.DataFrame(onehotencoded.toarray(), columns=features_of_one_hot)
one_hot_data

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


### Concating the two dataframes (Always first drop the unwanted column)

In [182]:
data.drop("Geography", axis = 1, inplace=True)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,42,2,0.00,1,1,1,101348.88,1
1,608,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,39,1,0.00,2,0,0,93826.63,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0


In [183]:
data = pd.concat([data, one_hot_data], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


### saving our encoder data in a pickle file which can be used later
Here 'wb' is write byte

In [184]:
with open('label_encoder.pkl', 'wb') as file:
    pickle.dump(encoder, file)
with open('one_hot_encoder.pkl', "wb") as file:
    pickle.dump(onehotencoder, file)

### train test split and standard scaler and storing scaler data in a pickle file

In [185]:
y = data[["Exited"]]
x = data.drop("Exited", axis = 1)

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=42)

In [186]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [187]:
with open('standart_scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## ANN implementation
(using tensorflow)

In [188]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [189]:
model = Sequential(
    [
        Dense(64, activation='relu', input_shape = (x_train.shape[1],)), #Hidden Layer 1
        Dense(16, activation='relu'), #Hidden Layer 2
        Dense(1, activation = 'sigmoid') #Hidden Layer 3
    ]
)

c:\Users\mayan\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [190]:
model.summary()

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_45 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 16)             │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,889 (7.38 KB)

 Trainable params: 1,889 (7.38 KB)

 Non-trainable params: 0 (0.00 B)

In [191]:
optimizer_to_use = tf.keras.optimizers.Adam(learning_rate=0.01)
loss_to_use = tf.keras.losses.BinaryCrossentropy()

In [192]:
model.compile(optimizer=optimizer_to_use, loss=loss_to_use, metrics=['accuracy'])

### Setting up Tensorboard

In [193]:
log_dir = "logs/fit"+datetime.datetime.now().strftime("%Y%M%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir, histogram_freq=1)

### Setting up Early stopping

In [194]:
early_stop_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

### Model training

In [195]:
training = model.fit(
    x_train, y_train, validation_data = (x_test, y_test), epochs = 100, callbacks = [tensorflow_callback, early_stop_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7896 - loss: 0.4586 - val_accuracy: 0.8570 - val_loss: 0.3587
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8458 - loss: 0.3696 - val_accuracy: 0.8555 - val_loss: 0.3428
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8632 - loss: 0.3375 - val_accuracy: 0.8570 - val_loss: 0.3409
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8588 - loss: 0.3396 - val_accuracy: 0.8560 - val_loss: 0.3542
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8566 - loss: 0.3476 - val_accuracy: 0.8545 - val_loss: 0.3474
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8682 - loss: 0.3228 - val_accuracy: 0.8595 - val_loss: 0.3441
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8700 - loss: 0.3163 - val_accuracy: 0.8605 - val_loss: 0.3570
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8662 - loss: 0.3303 - val_accu

### Launching the tensorboard

In [196]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [197]:
%tensorboard --log_dir logs/fit20254707-034703

Launching TensorBoard...

KeyboardInterrupt: 

### Making predictions using pickle and tensorboard files created

In [198]:
from tensorflow.keras.models import load_model

In [ ]:
loaded_model = load_model("model.h5")

with open("label_encoder.pkl", "rb") as file:
    loaded_label_encoder = pickle.load(file)

with open("one_hot_encoder.pkl", "rb") as file:
    loaded_one_hot_encoder = pickle.load(file)

with open("standart_scaler.pkl", "rb") as file:
    loaded_standard_scaler = pickle.load(file)


# Doing Regression

We will now estimate salary

In [199]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [200]:
x = data.drop("EstimatedSalary", axis = 1)
y = data["EstimatedSalary"]

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, random_state=42)

In [201]:
from tensorflow.keras.models import Sequential

In [207]:
model_reg = Sequential([
    Dense(64, activation = 'relu', input_shape = (x_train.shape[1],)),
    Dense(16, activation = 'relu'),
    Dense(1)
])

model_reg.compile(optimizer = 'adam', loss = "mean_absolute_error", metrics = ['mae'])

model.summary()

c:\Users\mayan\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_45 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 16)             │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,669 (22.15 KB)

 Trainable params: 1,889 (7.38 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,780 (14.77 KB)

In [205]:
early_stop_callback_reg = EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights = True)

In [208]:
train_reg = model_reg.fit(
    x_train, y_train, validation_data = (x_test, y_test), epochs = 100, callbacks = [early_stop_callback_reg]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 81501.5469 - mae: 81501.5469 - val_loss: 67708.0469 - val_mae: 67708.0469
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 68691.1719 - mae: 68691.1719 - val_loss: 65262.4492 - val_mae: 65262.4492
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 65633.6953 - mae: 65633.6953 - val_loss: 61034.3516 - val_mae: 61034.3516
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 59435.6367 - mae: 59435.6367 - val_loss: 54893.1758 - val_mae: 54893.1758
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 54494.5352 - mae: 54494.5352 - val_loss: 52001.3750 - val_mae: 52001.3750
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 51959.2617 - mae: 51959.2617 - val_loss: 50593.3203 - val_mae: 50593.3203
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 51316.5547 - mae: 51316.5547 - val_loss: 50395.4844 - val_mae: 50395.4844
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - 